In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from google.colab import drive

# 케이폴드~
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df=pd.read_pickle("/content/drive/My Drive/input/LSWMD.pkl")
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        811457 non-null  object 
 1   dieSize         811457 non-null  float64
 2   lotName         811457 non-null  object 
 3   waferIndex      811457 non-null  float64
 4   trianTestLabel  811457 non-null  object 
 5   failureType     811457 non-null  object 
dtypes: float64(2), object(4)
memory usage: 37.1+ MB


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
243595,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2683.0,lot15196,25.0,[],[]
61034,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2,...",710.0,lot4800,7.0,[],[]
215958,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1169.0,lot13704,22.0,[],[]
287578,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2367.0,lot17543,18.0,[],[]
226742,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7810.0,lot14254,2.0,[],[]


In [3]:
#각 웨이퍼 마다 크기를 waferMapDim에 저장
def find_dim(x):
    dim0=np.size(x,axis=0)
    dim1=np.size(x,axis=1)
    return dim0,dim1
df['waferMapDim']=df.waferMap.apply(find_dim)
df.sample(5)

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,waferMapDim
317179,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1534.0,lot19279,13.0,[],[],"(45, 43)"
128062,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1574.0,lot8417,25.0,[],[],"(51, 40)"
194556,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1930.0,lot12359,14.0,[],[],"(59, 42)"
449274,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",712.0,lot27561,24.0,[],[],"(32, 29)"
220940,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2793.0,lot13951,13.0,[],[],"(59, 60)"


In [4]:
#디펙타입을 숫자로 변형해 저장, test인지 train인지 trainTestNum에 저장
df['failureNum']=df.failureType
df['trainTestNum']=df.trianTestLabel
mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8} #매핑타입을 숫자로 변경
mapping_traintest={'Training':0,'Test':1}
df=df.replace({'failureNum':mapping_type, 'trainTestNum':mapping_traintest})

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/managers.py:1985: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


In [5]:
#디펙이 없거나 레이블링 안된 개수
df_withlabel = df[(df['failureNum']>=0) & (df['failureNum']<=8)]
df_withlabel =df_withlabel.reset_index()

#디펙있고 레이블링된 개수
df_withpattern = df[(df['failureNum']>=0) & (df['failureNum']<=7)]
df_withpattern = df_withpattern.reset_index()

#none으로 레이블링된 개수
df_nonpattern = df[(df['failureNum']==8)]
df_withlabel.shape[0], df_withpattern.shape[0], df_nonpattern.shape[0]

(172950, 25519, 147431)

In [6]:
sub_df = df.loc[df['waferMapDim'] == (26, 26)] # 모든 waferMapDim를 (26, 26)으로 변경해서 sub_df에 저장
sub_wafer = sub_df['waferMap'].values # waferMap 의 값을 뽑아서 sub_wafer에 저장

sw = np.ones((1, 26, 26))
label = list()

for i in range(len(sub_df)):
    # skip null label
    if len(sub_df.iloc[i,:]['failureType']) == 0: # 모든 sub_df에 대해 디펙타입이 0(레이블링이 안된 경우)인 경우 패스
        continue
    sw = np.concatenate((sw, sub_df.iloc[i,:]['waferMap'].reshape(1, 26, 26))) # sw에 각 waferMap을 (1,26,26)으로 저장
    label.append(sub_df.iloc[i,:]['failureType'][0][0]) # 디펙타입

In [7]:
sub_df.sample(5)

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,waferMapDim,failureNum,trainTestNum
8602,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,...",520.0,lot1626,20.0,[],[],"(26, 26)",[],[]
647805,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1,...",533.0,lot40657,17.0,[[Test]],[[none]],"(26, 26)",8,1
3435,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1,...",533.0,lot138,19.0,[[Training]],[[none]],"(26, 26)",8,0
6315,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 1, 1,...",533.0,lot254,18.0,[[Training]],[[none]],"(26, 26)",8,0
5539,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1,...",533.0,lot223,17.0,[[Training]],[[none]],"(26, 26)",8,0


In [8]:
x = sw[1:]
y = np.array(label).reshape((-1,1))

In [9]:
x = x.reshape((-1, 26, 26, 1))

In [10]:
# 정렬
faulty_case = np.unique(y)
print('Faulty case list : {}'.format(faulty_case))

Faulty case list : ['Center' 'Donut' 'Edge-Loc' 'Edge-Ring' 'Loc' 'Near-full' 'Random'
 'Scratch' 'none']


In [11]:
for f in faulty_case :
    print('{} : {}'.format(f, len(y[y==f])))

Center : 90
Donut : 1
Edge-Loc : 296
Edge-Ring : 31
Loc : 297
Near-full : 16
Random : 74
Scratch : 72
none : 13489


In [12]:
new_x = np.zeros((len(x), 26, 26, 3))

for w in range(len(x)):
    for i in range(26):
        for j in range(26):
            new_x[w, i, j, int(x[w, i, j])] = 1

In [13]:
new_x.shape

(14366, 26, 26, 3)

In [14]:
from keras import Input, models
##이미지 제너레이터
# Encoder
input_shape = (26, 26, 3)
input_tensor = Input(input_shape)
encode = layers.Conv2D(64, (3,3), padding='same', activation='relu')(input_tensor)

latent_vector = layers.MaxPool2D()(encode)

# Decoder
decode_layer_1 = layers.Conv2DTranspose(64, (3,3), padding='same', activation='relu')
decode_layer_2 = layers.UpSampling2D()
output_tensor = layers.Conv2DTranspose(3, (3,3), padding='same', activation='sigmoid')

# connect decoder layers
decode = decode_layer_1(latent_vector)
decode = decode_layer_2(decode)

ae = models.Model(input_tensor, output_tensor(decode))
ae.compile(optimizer = 'Adam',
              loss = 'mse',
             )

epoch=30
batch_size=1024

The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add), but
are not present in its tracked objects:
  <tf.Variable 'conv2d/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.conv2d_transpose), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_transpose/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
It is possible that this is intended

In [15]:
# 이미지 제너레이팅 시작 start train
ae.fit(new_x, new_x,
       batch_size=batch_size,
       epochs=epoch,
       verbose=2)

Epoch 1/30
15/15 - 41s - loss: 0.2484
Epoch 2/30
15/15 - 0s - loss: 0.2484
Epoch 3/30
15/15 - 0s - loss: 0.2484
Epoch 4/30
15/15 - 0s - loss: 0.2484
Epoch 5/30
15/15 - 0s - loss: 0.2484
Epoch 6/30
15/15 - 0s - loss: 0.2484
Epoch 7/30
15/15 - 0s - loss: 0.2484
Epoch 8/30
15/15 - 0s - loss: 0.2484
Epoch 9/30
15/15 - 0s - loss: 0.2484
Epoch 10/30
15/15 - 0s - loss: 0.2484
Epoch 11/30
15/15 - 0s - loss: 0.2484
Epoch 12/30
15/15 - 0s - loss: 0.2484
Epoch 13/30
15/15 - 0s - loss: 0.2484
Epoch 14/30
15/15 - 0s - loss: 0.2484
Epoch 15/30
15/15 - 0s - loss: 0.2484
Epoch 16/30
15/15 - 0s - loss: 0.2484
Epoch 17/30
15/15 - 0s - loss: 0.2484
Epoch 18/30
15/15 - 0s - loss: 0.2484
Epoch 19/30
15/15 - 0s - loss: 0.2484
Epoch 20/30
15/15 - 0s - loss: 0.2484
Epoch 21/30
15/15 - 0s - loss: 0.2484
Epoch 22/30
15/15 - 0s - loss: 0.2484
Epoch 23/30
15/15 - 0s - loss: 0.2484
Epoch 24/30
15/15 - 0s - loss: 0.2484
Epoch 25/30
15/15 - 0s - loss: 0.2484
Epoch 26/30
15/15 - 0s - loss: 0.2484
Epoch 27/30
15/15 - 

In [16]:
encoder = models.Model(input_tensor, latent_vector)

decoder_input = Input((13, 13, 64))
decode = decode_layer_1(decoder_input)
decode = decode_layer_2(decode)

decoder = models.Model(decoder_input, output_tensor(decode))

# Encode original faulty wafer
encoded_x = encoder.predict(new_x)

# Add noise to encoded latent faulty wafers vector.
noised_encoded_x = encoded_x + np.random.normal(loc=0, scale=0.1, size = (len(encoded_x), 13, 13, 64))

The following Variables were used a Lambda layer's call (tf.nn.conv2d_transpose_2), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_transpose/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add_3), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_transpose/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.conv2d_transpose_3), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_transpose_1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It 

In [17]:
# augment function define
def gen_data(wafer, label):
    # Encode input wafer
    encoded_x = encoder.predict(wafer)
    
    # dummy array for collecting noised wafer
    gen_x = np.zeros((1, 26, 26, 3))
    
    # Make wafer until total # of wafer to 2000
    for i in range((2000//len(wafer)) + 1):
        noised_encoded_x = encoded_x + np.random.normal(loc=0, scale=0.1, size = (len(encoded_x), 13, 13, 64)) 
        noised_gen_x = decoder.predict(noised_encoded_x)
        gen_x = np.concatenate((gen_x, noised_gen_x), axis=0)
    # also make label vector with same length
    gen_y = np.full((len(gen_x), 1), label)
    
    # return date without 1st dummy data.
    return gen_x[1:], gen_y[1:]

In [18]:
# Augmentation for all faulty case.
for f in faulty_case : 
    # skip none case
    if f == 'none' : 
        continue
    
    gen_x, gen_y = gen_data(new_x[np.where(y==f)[0]], f)
    new_x = np.concatenate((new_x, gen_x), axis=0)
    y = np.concatenate((y, gen_y))


In [19]:
for f in faulty_case :
    print('{} : {}'.format(f, len(y[y==f])))

Center : 2160
Donut : 2002
Edge-Loc : 2368
Edge-Ring : 2046
Loc : 2376
Near-full : 2032
Random : 2146
Scratch : 2088
none : 13489


In [20]:
none_idx = np.where(y=='none')[0][np.random.choice(len(np.where(y=='none')[0]), size=11000, replace=False)]

new_x = np.delete(new_x, none_idx, axis=0)
new_y = np.delete(y, none_idx, axis=0)

In [21]:
for f in faulty_case :
    print('{} : {}'.format(f, len(new_y[new_y==f])))

Center : 2160
Donut : 2002
Edge-Loc : 2368
Edge-Ring : 2046
Loc : 2376
Near-full : 2032
Random : 2146
Scratch : 2088
none : 2489


In [22]:
from keras.utils import to_categorical
for i, l in enumerate(faulty_case):
    new_y[new_y==l] = i
    
# one-hot-encoding
new_y = to_categorical(new_y)

new_X=new_x[0:19000] # x가 웨이퍼고, y가 labeling
new_Y=new_y[0:19000] 
test_x=new_x[19001:19706]
test_y=new_y[19001:19706]
test_x.shape

ImportError: ignored

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y,
                                                    test_size=0.33,
                                                    random_state=2019)

In [ ]:
print('Train x : {}, y : {}'.format(x_train.shape, y_train.shape))
print('Test x: {}, y : {}'.format(x_test.shape, y_test.shape))

In [ ]:
print("{}".format(x_train[2]))

In [ ]:
print("{}".format(y_train[3]))

In [ ]:
print("{}", df['waferMap'][3004])

In [ ]:
# train_set = '/content/drive/My Drive/input/웨이퍼맵1/train'
# test_set = '/content/drive/My Drive/input/웨이퍼맵1/test'

# imageGenerator = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     brightness_range=[.2, .2],
#     horizontal_flip=True,
#     validation_split=.1,
#     fill_mode='nearest'
# )

# trainGen = imageGenerator.flow_from_directory(
#     train_set,
#     target_size=(64, 64),
#     color_mode='rgb',
#     subset='training',
#     class_mode='categorical'
# )

# validationGen = imageGenerator.flow_from_directory(
#     test_set,
#     target_size=(64, 64),
#     color_mode='rgb',
#     subset='validation',
#     class_mode='categorical',
#     shuffle=False
# )


model = Sequential()


model.add(layers.InputLayer(input_shape=(26, 26, 3)))
model.add(layers.Conv2D(filters=16, kernel_size=(3,3),input_shape=(26, 26, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))



# model.summary()

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

filename = '/content/drive/My Drive/model_file_simpleCNN/checkpoint_CNN-{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.h5' # 가중치를 저장할 파일
# checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
#                              monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
#                              verbose=1,            # 로그를 출력합니다
#                              save_best_only=True,  # 가장 best 값만 저장합니다
#                              mode='auto'           # auto는 알아서 best를 찾습니다. min/max
#                             )

history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=1024,
    shuffle=True,
    validation_data=(x_test, y_test)
    # callbacks=[checkpoint]
)

model.save("/content/drive/My Drive/model_file_simpleCNN/checkpoint_CNN-{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.h5")

# 3-Fold Crossvalidation
# kfold = KFold(n_splits=3, shuffle=True, random_state=2019) 
# results = cross_val_score(history, x_train, y_train, cv=kfold)